In [1]:
from typing import List
from IPython.display import display, Math, Latex

def bb(n, bits=7):
    mask = (1 << bits) - 1
    if n < 0:
        n = ((abs(n) ^ mask) + 1)
    return ('{0:0' + str(bits) + 'b}').format(n & mask)
  
def bn(n, bits=7, underline=None, sep=None):
    mask = (1 << bits) - 1
    if n < 0:
        n = ((abs(n) ^ mask) + 1)
    arr = list(('{0:0' + str(bits) + 'b}').format(n & mask))
    
    arr.reverse()
    if underline is not None: arr[underline] = r'\underline{' + arr[underline] + '}'
    if sep is not None: arr.insert(sep, '|')
    arr.reverse()
    return '\ '.join(arr)
 

def text(s: str) -> str:
  s = s.replace(' ', '\ ')
  s = r'\textup{' + s + '}'
  return s

def let(a: str, sign, sub) -> str:
  r = r' \left[' + ('-' if sign else '') + a + r'\right]_{\textup{' + sub + r'}}'
  return r

def arr(l: List[str], t='c') -> str:
  r = r'\begin{array}{' + t + r'}'
  r += r' \\ '.join(l)
  r += r'\end{array}'
  return r

def fix(val, digits=16):
  # return int(bb(val, digits), 2)
  val &= ((1<<digits)-1)
  if val & (1<<(digits - 1)): val -= (1<<digits)
  return val

def sign(n: int):
  if n < 0: return 1
  return 0

class Div():
  table: List[List[str]]
  
  def __init__(self, a: int, b: int, aw: int, bw: int) -> None:
    self.a = a
    self.b = b
    self.aw = aw
    self.bw = bw
    self.table = [[
        arr([
          text('N'),
          text('шага')
        ]),
        arr([
          text('Операнды '),
          text('и действия')
        ]),
        arr([
          text('Делимое/остаток'),
          text('старш. разряды')
        ]),
        arr([
          text('Делимое/остаток'),
          text('младш. разряды')
        ])
    ]]
    self.top = ''
    self.bot = ''
    self.process(a, b, aw=aw, bw=bw)
  
  def __str__(self) -> str:
    return self.print()
    
  def print(self):
    la = r'[A]_{пр.}' if self.a > 0 else  r'[-A]_{доп.}'
    lb = r'[B]_{пр.}' if self.b > 0 else  r'[-B]_{доп.}'
    la += '=' + bb(self.a)
    lb += '=' + bb(self.b)
    self.top = (
      '(A' + ('>' if self.a > 0 else '<') + '0,\ B' + ('>' if self.b > 0 else '<') + '0)')
    self.top += r' \\ ' + '\n'
    self.top += '\left.' + la + ';\ ' + lb + '\\right.'
    
    res = r'\begin{array}{c}'
    res += self.top
    res += r' \\ '  + '\n' + r' \\ '
    res += (
      r'\begin{array}{|c|c|c|c|} \hline ' +
      (r' \\ \hline ' + '\n').join(' & '.join(row) for row in self.table) +
      r' \\ \hline ' + '\n' + r' \end{array} \\' + '\n'
      )
    res += r' \\ ' + '\n' + r' \\ '
    res += self.bot
    res += r'\end{array}'
    
    return f'$${res}$$'
    # display(Math(res))
    
  def process(self, a: int, b: int, aw: int, bw: int):
    step = 0
    r = a
    # if sign(a) == sign(b):
    r = a + b
    # else:
      # r = a - b
    # r2 = r + b
    self.table.append([
      str(step),
      arr([
        let('A', 0, ''),
        let('B', 0, ''),
        let('R_1', 0, ''),
      ]),
      arr([
        bn(a >> (aw//2), aw//2),
        bn(b>>bw, bw),
        bn(r>>bw, aw//2),
      ]),
      arr([
        bn(a, aw//2),
        bn(b, bw),
        bn(r, bw),
      ])
      ])
    # r = r2
    
    while True:
      step += 1
      r <<= 1
      if bb(r, 16)[0] != bb(b, bw)[0]:
        t = b << bw
      else:
        t = -b << bw
      r2 = (r + t)
      d = (sign(b) == sign(r2)) * 1
      r2 |= d
      self.table.append([
      str(step),
      arr([
        f'\overleftarrow{{R}}_{step-1}',
        let('B', 0, ''),
        f'R_{step}'
        ]),
      arr([
        bn(r >> (aw//2), aw//2),
        bn(t>>bw, bw),
        bn(r2>>bw, aw//2),
      ]),
      arr([
        bn(r, aw//2, sep=step),
        '',
        bn(r2, aw//2, sep=step, underline=0),
      ])
      ])
      r = r2
      if step == bw:
        self.table.append([
            text('Коррекция остатка'),
            arr([
              'R'
            ]),
            arr([
              bn(r >> bw , aw//2),
            ]),
            arr([
              bn(r, aw//2, sep=step),
            ])
            ])
        if b > 0:
          r -= 1
        self.table.append([
            text('Коррекция частного'),
            arr([
              'R'
            ]),
            arr([
              bn(r >> bw , aw//2),
            ]),
            arr([
              bn(r, aw//2, sep=step),
            ])
            ])
        break
    self.bot = ''
    if a // b > 0:
      self.bot += f"{let('C', 0, 'пр')}={bb(r, bw)}_2={fix(int(bb(r, bw),     2), bw)}_{{10}}" 
    else:
      self.bot += f"{let('C', 0, 'доп')}={bb(r, bw)}_2\\\\"
      self.bot += f"{let('C', 0, 'пр')}=-{bb(-r, bw)}_2="
      self.bot += f"{fix(int(bb(r, bw),     2), bw)}_{{10}}" 
      
    self.bot += ' \\\\ '
    print(a % b)
    if r>>bw >= 0:
      self.bot += f"{let('R', 0, 'пр')}={bb(r>>bw, bw)}_2={fix(int(bb(r>>bw, bw),     2), bw)}_{{10}}" 
    else:
      self.bot += f"{let('R', 0, 'доп')}={bb(r>>bw, bw)}_2\\\\"
      self.bot += f"{let('R', 0, 'пр')}=-{bb(-(r>>bw), bw)}_2="
      self.bot += f"{fix(int(bb(r>>bw, bw),     2), bw)}_{{10}}" 
      
    print('res', bb(r, bw),     fix(int(bb(r, bw),     2), bw))
    print('rem', bb(r>>bw, bw), fix(int(bb(r>>bw, bw), 2), bw))
    # self.bot = r' \left[C\right]_{\textup{пр.}}' if a > 0 else  r' \left[C\right]_{\textup{доп}.}'
    # self.bot += '='
    # self.bot += bb( a * b, aw * 2) + '_2 = '
    # self.bot += str(a * b) + '_{10}'
    
open('res.tex', 'w+', encoding='utf-8').write(f'''
  {Div(1150, 23, aw=16, bw=8)}
  {Div(-1150, 23, aw=16, bw=8)}
  {Div(1150, -23, aw=16, bw=8)}
  {Div(-1150, -23, aw=16, bw=8)}
''')

# open('res.tex', 'w+', encoding='utf-8').write(f'''
#   {Div(139, 13, aw=16, bw=8)}
#   {Div(-139, 13, aw=16, bw=8)}
#   {Div(139, -13, aw=16, bw=8)}
#   {Div(-139, -13, aw=16, bw=8)}
# ''')

# open('res.tex', 'w+', encoding='utf-8').write(f'''
#    {Div(139, -13, aw=10, bw=5)}
#    {Div(-139, 13, aw=10, bw=5)}
#    {Div(-139, -13, aw=10, bw=5)}
#  ''')

0
res 00110010 50
rem 00000000 0
0
res 11001110 -50
rem 00000000 0
0
res 11001110 -50
rem 00000000 0
0
res 00110010 50
rem 00000000 0


14887